In [4]:
import pandas as pd


df_loyaty = pd.read_csv("loyalty_data.csv",encoding="latin-1")
df_cc = pd.read_csv("cc_data.csv",encoding="latin-1")

print(df_loyaty.head())
print(df_cc.head())

    timestamp             location  price loyaltynum
0  01/06/2014  Brew've Been Served   4.17      L2247
1  01/06/2014  Brew've Been Served   9.60      L9406
2  01/06/2014     Hallowed Grounds  16.53      L8328
3  01/06/2014         Coffee Shack  11.51      L6417
4  01/06/2014     Hallowed Grounds  12.93      L1107
          timestamp             location  price  last4ccnum
0  01/06/2014 07:28  Brew've Been Served  11.34        4795
1  01/06/2014 07:34     Hallowed Grounds  52.22        7108
2  01/06/2014 07:35  Brew've Been Served   8.33        6816
3  01/06/2014 07:36     Hallowed Grounds  16.72        9617
4  01/06/2014 07:37  Brew've Been Served   4.24        7384


In [2]:
import plotly.graph_objects as go
import plotly.express as px
import plotly.subplots as sp
import pandas as pd

df_cc = pd.read_csv("cc_data.csv",parse_dates=['timestamp'],encoding="latin-1")

df_cc['day'] = df_cc['timestamp'].dt.strftime('%Y-%m-%d')
df_cc['hour'] = df_cc['timestamp'].dt.hour

grouped_data = df_cc.groupby(['location', 'day', 'hour']).size().reset_index(name='visitor_count')


unique_days = sorted(grouped_data['day'].unique())
print(unique_days)



['2014-01-06', '2014-01-07', '2014-01-08', '2014-01-09', '2014-01-10', '2014-01-11', '2014-01-12', '2014-01-13', '2014-01-14', '2014-01-15', '2014-01-16', '2014-01-17', '2014-01-18', '2014-01-19']
